In [1]:
import pytesseract
import requests
import imutils
import cv2
from bs4 import BeautifulSoup
import numpy as np

In [3]:
def details(regNo):
    with requests.Session() as s:
        url='http://vahan.nic.in/nrservices/faces/user/searchstatus.xhtml'
        header={
              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
                }
        dat=s.get(url,headers=header)
        cap_url='http://vahan.nic.in/nrservices/cap_img.jsp'
        f = open('captcha.jpeg','wb')
        f.write(s.get(cap_url).content)
        f.close()
        img=cv2.imread('captcha.jpeg')
        text=pytesseract.image_to_string(img).upper()
        sp=BeautifulSoup(dat.content,'html')
        viewState=sp.find('input', {'id':"j_id1:javax.faces.ViewState:0"}).get('value')
        jfs=sp.find('button',{'class':'ui-button'}).get('id')
        data={
            'javax.faces.partial.ajax': 'true',
            'javax.faces.source': 'j_idt47',
            'javax.faces.partial.execute': '@all',
            'javax.faces.partial.render': 'rcDetailsPanel resultPanel userMessages capatcha txt_ALPHA_NUMERIC',
            'masterLayout': 'masterLayout',
            jfs: jfs,
#             'j_idt37': '',
            'regn_no1_exact': regNo,
            'txt_ALPHA_NUMERIC': text,
            'javax.faces.ViewState': viewState,
            }
        final=s.post(url,data=data,headers=header)
        result=BeautifulSoup(final.content,'html')
        dt=result.find('div',{'class':'col-md-12'})
        er=result.find('error')
#         print(result)
        if dt:
#             print(result)
            info=[]
            rows=result.find_all('div',{'class':'row'})
            for tr in rows:
                colu=tr.find_all('div')
                for c in colu:
                    st=c.text.strip()
                    info.append(st+'\n')
            return info
        elif er:
            return list("Sorry, You've entered wrong vehicle number. Please Try again")
        else:
#             return [text]
            return details(regNo)
image = cv2.imread('car1.jpg')
image = imutils.resize(image, width=500)
cv2.imshow("Original Image", image)
cv2.waitKey(0)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Grays", gray)
cv2.waitKey(0)
gray = cv2.bilateralFilter(gray, 11, 17, 17)
cv2.imshow("BiFilter", gray)
cv2.waitKey(0)
edged = cv2.Canny(gray, 170, 200)
cv2.imshow("Edges", edged)
cv2.waitKey(0)
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] 
for c in cnts:
        peri = cv2.arcLength(c, True)
        corn = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(corn) == 4:  
            plate = corn
            break
nw=image.copy()
cv2.drawContours(nw, [plate], -1, (0,255,0), 3)
cv2.imshow("Marked Plate",nw)
cv2.waitKey(0)
x,y,w,h=cv2.boundingRect(plate)
fin_img=image[y:y + h,x:x+w]
cv2.imshow("Separated Plate",fin_img)
cv2.waitKey(0)
fin_img=cv2.cvtColor(fin_img,cv2.COLOR_BGR2GRAY)
cv2.destroyAllWindows()
text=pytesseract.image_to_string(fin_img)
text=text.replace(" ","")
text=text.replace('.',"")
text=text.replace(',',"")
text=text.replace('-',"")
print(text)
print(*details(text))


KL65K5455
Vehicle Details Showing in Registering Authority
 Vehicle Details Showing in Registering Authority
 1. Registering Authority:  THIRURANGADI SRTO, Kerala
 1. Registering Authority:  THIRURANGADI SRTO, Kerala
 Registration No:
 KL65K5455
 Registration Date:
 06-Nov-2017
 Chassis No:
 MCAAJPBY6JFA07****H
 Engine No:
 40****6
 Owner Name:
 ABDUL ASSEES VETTILAKKARAN
 Vehicle Class:
 Motor Car(LMV)
 Fuel:
 DIESEL
 Maker / Model:
 FCAIAP FCA INDIA AUTOMOBILES /  JEEP COMPASS LONGITUDE O
 Fitness/REGN Upto:
 05-Nov-2032
 MV Tax upto:
 30-Sep-2032
 Insurance Upto:
 18-Oct-2020
 PUCC Upto:
 NA
 Emission norms:
 Not Available
 RC Status:
 ACTIVE
 Financed:
 YES 

click here to see CERSAI details
 BlackList Status:
 Blacklisted by THIRURANGADI SRTO,Kerala due to reason Others, psm

